In [1]:
%cd "~/projects/spartDM"
%load_ext autoreload
%autoreload 2

import os

print(f"Current working dir: {os.getcwd()}")

/home/kdang/projects/spartDM
Current working dir: /home/kdang/projects/spartDM


/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Init model

In [4]:
import torch
from einops import rearrange

device = torch.device("cpu")


In [3]:
# Trained encoder from representation-learning

# CondDDPM model

from src.ldae import CondDDPM
from lightning.pytorch.utilities.model_summary import ModelSummary


WORK_DIR = "workdir/diffae_starmen"
# WORK_DIR = "workdir/debug"

## Load model
ckpt_path = os.path.join(WORK_DIR, "representation-learning", "checkpoints", "best.ckpt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

diffae_litmodel = CondDDPM.load_from_checkpoint(ckpt_path, map_location=device)
encoder = diffae_litmodel.ema_encoder
encoder.eval()
ema_decoder = diffae_litmodel.ema_decoder
ema_decoder.eval()
gaussian_diffusion = diffae_litmodel.gaussian_diffusion

# Model summary

summary = ModelSummary(diffae_litmodel, max_depth=1)
print(summary)

/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/generative/networks/layers/vector_quantizer.py:86: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/generative/networks/layers/vector_quantizer.py:124: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


Backbone model loaded successfully.
Frozen first 0 layers out of 161 in the backbone

  | Name        | Type            | Params | Mode 
--------------------------------------------------------
0 | decoder     | CondUNet        | 2.5 M  | train
1 | encoder     | SemanticEncoder | 24.6 M | train
2 | ema_encoder | SemanticEncoder | 24.6 M | eval 
3 | ema_decoder | CondUNet        | 2.5 M  | eval 
--------------------------------------------------------
27.1 M    Trainable params
27.1 M    Non-trainable params
54.1 M    Total params
216.547   Total estimated model params size (MB)
492       Modules in train mode
492       Modules in eval mode


In [131]:
# Longitudinal Transformer model
from src.ldae.nets.longitudinal_transformer import LongTransformer

import argparse

lt_args = argparse.Namespace(
    attn_map=True,
    max_seq_len=10, 
    n_heads=8,
    n_layers=2,
    dropout=0.0,
    learned_pe=False, 
    tpe=True,
    step_ahead=True
)

long_transformer = LongTransformer(encoder=encoder, args=lt_args, device=device)

## Toy dataloader

In [22]:
from src.data.datamodules import StarmenDataset
from monai.data import DataLoader
from monai.data.utils import first

train_ds = StarmenDataset(
    data_dir="data/starmen/output_random_noacc",
    split="train",
    nb_subject=None,
)


train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=1)

test_batch = first(iter(train_loader))

In [38]:
x_org = test_batch["x_origin"]
x_org.shape

age = test_batch["age"]
age.shape

torch.Size([2, 10])

## Forward pass

In [133]:
# prepare input
seq_lengths = [10 for i in range(x_org.shape[0])]
seq_lengths

x = rearrange(x_org, "b t ... -> (b t) ...")
x.shape

torch.Size([20, 1, 64, 64])

In [134]:
self = long_transformer

# Forward pass
embeddings = self.encoder(x)

# Reshape embeddings to batch_size x seq_length x n_features
embeddings = embeddings.reshape(len(seq_lengths), self.args.max_seq_len, self.encoder.embedding_dim)
embeddings.shape

if self.args.tpe:
    x = self.pos_encoder(embeddings, age)
else:
    x = self.pos_encoder(embeddings, None)


In [135]:
# Create mask to ignore padding tokens. For each sequence of visits, mask all tokens beyond last visit
# Here, 1 = pad (ignore), 0 = valid (keep)
src_key_padding_mask = torch.ones((x.shape[0], x.shape[1])).float().to(self.device)  # batch x seq_length
for i, seq_length in enumerate(seq_lengths):
    src_key_padding_mask[i, :seq_length] = 0

# Transformer modeling with "decoder-style" causal attention (only attend to current + PRIOR elements of each sequence)
feats, attn_map = self.transformer(x, mask=self.causal_mask, src_key_padding_mask=src_key_padding_mask, is_causal=True, need_weights=self.args.attn_map)


In [149]:
rel_times = age

if self.args.step_ahead:
    # Get time elapsed (delta) between consecutive visits
    delta_times = torch.diff(rel_times)  # batch x max_seq_len-1
    delta_times[delta_times < 0] = 0
    delta_times = torch.nn.functional.pad(delta_times, (0, 1), 'constant', 0)  # batch x max_seq_len

    # Use relative temporal timestep encoding to inform the model of "# year of into the future for which to predict imaging features"
    delta_encoded_feats = self.rel_pos_encoder(feats, delta_times) 

    # Predict imaging features of *next* visit for each subsequence
    feat_preds = self.step_ahead_predictor(delta_encoded_feats)

    # Get actual imaging features of next visit
    feat_targets = torch.nn.functional.pad(feats[:, 1:, :], (0, 0, 0, 1), 'constant', 0)

feat_preds.shape

torch.Size([2, 10, 512])